<h3>DATA SCIENCE INTERVIEW CHALLENGE<h3>

For this exercise, you will analyze a dataset from Amazon. The data format and a sample entry are shown on the next page.
A. (Suggested duration: 90 mins)
With the given data for 548552 products, perform exploratory analysis and make suggestions for further analysis on the following aspects.
1. Trustworthiness of ratings <br>
Ratings are susceptible to manipulation, bias etc. What can you say (quantitatively speaking) about the ratings in this dataset?
<br>
<b> It is using Likert scale (1-5) which a numerical value (a quantitative value) is assigned to a choice (qualitative). About 140K out of 542k of data have zero reviews so those needs to be cleaned or removed before getting the average of the ratings.  The average rating is 4.3 which is interpreted as "Good to Very Good" compared to 3.2 ("Neutral") for uncleaned data. <br>
The reason for not including items with no reviews is because averaging is much affected by extreme values (like zero). Thus, filtering items with no reviews increased the overall average rating from 3.2 ("Neutral") to 4.3 ("Good to Very Good").
<br> Software and Baby products have the highest average ratings at 4.5 ("Very Good") while Video games has the lowest rating of 2.5 ("Average".
</b>
2. Category bloat<br>
Consider the product group named 'Books'. Each product in this group is associated with categories. Naturally, with categorization, there are tradeoffs between how broad or specific the categories must be.
For this dataset, quantify the following:<br>
a. Is there redundancy in the categorization? How can it be identified/removed?
<br>
<b> Yes, some of the categories are duplicated like below since Preaching and Sermons can be combined together.
</b>
<br>
|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]<br>
|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]<br>

b. Is is possible to reduce the number of categories drastically (say to 10% of existing categories) by sacrificing relatively few category entries (say close to 10%)?
<br>
<b> Yes, it is possible to reduce the number of categories by filtering relatively few category entries.  We can use Machine Learning algorithm like Naive Bayes, SVM or NLTK or GenSim (uses cosine similarity) to classify similar words in the categories and combine them into fewer categories.
</b>


In [1]:
%matplotlib inline
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
from pylab import plot, show, text
import datetime
import matplotlib.dates as mdates
import pylab

In [2]:
# Extract all items and their average rating then save as a file 

i = 0
with open('amazon_users.txt', 'w', encoding="utf8") as file:
    with open('amazon-meta.txt', 'r', encoding="utf8") as f:
        for line in f:
            line = line.strip()
            #if i == 10:
            #    break
            if line[:3] == 'Id:':
                id = int(line[4:])
                i += 1
            if line[:5] == 'ASIN:':
                asin = line[6:]
            if line[:6] == 'title:':# and asin == '0312254040':
                title = line[7:].replace('"','').replace(',','')
            if line[:6] == 'group:':
                group = line[7:]
            if line[:8] == 'similar:':
                similar = int(line[9:11])
            if line[:11] == 'categories:':
                categories = (line[12:])
            if line[:8] == 'reviews:':# and asin == '0312254040':
                reviews = int(line[15:18])
                avg_rating = float(line[-3:].replace(':',''))
                strwrite = str(id) + ',' + asin + ',"' + title + '",' + group + ',' +  str(similar) + ',' +  categories + ',' +  str(reviews) + ',' +  str(avg_rating)
                file.write(strwrite + '\n')
file.close
         
            

<function TextIOWrapper.close>

In [9]:
#Display and write the categories of sample products

i, is_categories, is_group = 0, False, False
with open('amazon_categories.txt', 'w', encoding="utf8") as file:
    with open('amazon-meta.txt', 'r', encoding="utf8") as f:
        for line in f:
            line = line.strip()
            #if i == 10:
            #    break
            if line[:3] == 'Id:':
                id = int(line[4:])
            if line[:6] == 'group:':
                #print ('group', (line[7:]))
                is_group = line[7:] == 'Book'
                j = 0
                is_categories = False
                i += 1
                
            if line[:11] == 'categories:' and is_group:
                #print ('categories', (line[12:]))
                num_categories = int(line[12:])
                is_categories = True
            
            if (is_categories and is_group and j <= num_categories):
                j += 1
                if line[1:6] == 'Books':
                    strwrite = str(id) + ',' + str(line)
                    file.write(strwrite + '\n')
file.close
    

<function TextIOWrapper.close>

In [4]:
# read file and load to dataframe
df = pd.read_csv('amazon_users.txt', header=None, 
                 names=['id', 'asin', 'title', 'group', 'similar', 'categories', 'reviews', 'avg_rating']) 

In [5]:
df.describe()

,id,similar,categories,reviews,avg_rating
count,542684.000000,542684.000000,542684.000000,542684.000000,542684.000000
mean,274414.212208,3.296071,4.624605,7.498229,3.209534
std,158454.479276,2.287289,4.450647,13.759132,1.996296
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,137161.750000,0.000000,2.000000,0.000000,0.000000
50%,274427.500000,5.000000,4.000000,2.000000,4.000000
75%,411674.250000,5.000000,6.000000,8.000000,5.000000
max,548551.000000,5.000000,116.000000,99.000000,5.000000


In [19]:
# what item has the most number of reviews?
df[df['reviews'] == 99].group.value_counts()
# Answer: 180 items have 99 reviews 
# Book and Music groups are the most reviewed items

Book     104
Music     40
DVD       19
Video     19
Name: group, dtype: int64

In [7]:
# how many items that has no review
print(len(df[df.reviews == 0]))

139949


In [15]:
# which group has most no reviews
df_no_reviews = df[df.reviews == 0]
df_no_reviews.group.value_counts()
# answer: Book

Book     115337
Music     19481
Video      3774
DVD        1354
CE            2
Toy           1
Name: group, dtype: int64

In [8]:
# select items with reviews
df_avg_rating = df[df.reviews > 0]
df_avg_rating.avg_rating.describe()

count    402735.000000
mean          4.324836
std           0.739279
min           1.000000
25%           4.000000
50%           4.500000
75%           5.000000
max           5.000000
Name: avg_rating, dtype: float64

In [9]:
# what is the average rating of items per group?
#CE is consumer electronics"
df_avg_rating.groupby(['group']).avg_rating.mean().sort_values(ascending=False)

# Answer: Software and Baby Product groups have the highest average ratings at 4.5 
#    while Consumer Electronics and Video Games have the lowest average ratings at 3.5 and 2.5 respectively

group
Software        4.500000
Baby Product    4.500000
Music           4.482065
Toy             4.357143
Book            4.315994
Video           4.164579
Sports          4.000000
DVD             3.940051
CE              3.500000
Video Games     2.500000
Name: avg_rating, dtype: float64

#BELOW CELLS WILL WORK ON CATEGORY BLOAT

In [50]:
# read file and load to dataframe
df = pd.read_csv('amazon_categories.txt', header=None, names=['id', 'categories']) 

# get last category 
df['start_category'] = df['categories'].apply(lambda x: x[:x.rfind('|')])
df['end_category']   = df['categories'].apply(lambda x: x[x.rfind('|')+1:])
df.head(10)



,id,categories,start_category,end_category
0,1,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Preaching[12368]
1,1,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Sermons[12370]
2,2,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Wicca[12484]
3,2,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Witchcraft[12486]
4,3,|Books[283155]|Subjects[1000]|Home & Garden[48...,|Books[283155]|Subjects[1000]|Home & Garden[48...,General[5144]
5,4,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,New Testament[12159]
6,4,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Discipleship[12335]
7,4,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Life Application[572080]
8,4,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,New Testament[572082]
9,4,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Study Guides


In [121]:
# get all books with almost the same categories except last category
df_dup = df.groupby(['id', 'start_category']).filter(lambda x: len(x) > 1)
 
df_dup.head()

,id,categories,start_category,end_category
0,1,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Preaching[12368]
1,1,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Sermons[12370]
2,2,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Wicca[12484]
3,2,|Books[283155]|Subjects[1000]|Religion & Spiri...,|Books[283155]|Subjects[1000]|Religion & Spiri...,Witchcraft[12486]
18,8,|Books[283155]|Subjects[1000]|Nonfiction[53]|P...,|Books[283155]|Subjects[1000]|Nonfiction[53]|P...,General[11083]


In [123]:
# clean end_category and remove non-chars
import re
df_dup['end_cat'] = df_dup['end_category'].apply(lambda x: re.sub(r'[^a-zA-Z +]', '', x))



In [135]:

# remove NaN values; some category are numeric like 79 or 99 (why? is it input error?)
df_dup['end_cat'].replace('', np.nan, inplace=True)
df_dup.dropna(subset=['end_cat'], inplace=True)   

# reset index of new dataframe
df_dup = df_dup.reset_index(drop=True)

In [136]:
# convert to lowercase since word2vec expects it
df_dup['end_cat'] = df_dup['end_cat'].str.lower()



In [139]:
# how many percent of categories can be combined using word2Vec model?
len(df_dup.id.value_counts())/len(df)*100

# answer: 12% (more than 10%!)

12.713467324812255

In [141]:
# save dataframe into a file

with open('amazon_bow_categories.txt', 'w', encoding="utf8") as file:
    for i in range(len(df_dup)):
        strwrite = str(df_dup.id[i]) + ',' + str(df_dup.end_cat[i])
        file.write(strwrite + '\n')
file.close


<function TextIOWrapper.close>

In [8]:
#load the model
#download text8 zip from website: https://cs.fit.edu/~mmahoney/compression/textdata.html
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Text8Corpus
import logging



In [ ]:
#display logs when loading word2vec text corpus
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus('text8')
model = word2vec.Word2Vec(sentences, size=200)


In [12]:
# use nltk lemmatizer to convert words into their normalized form
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#load categories into dataframe
df = pd.read_csv('amazon_bow_categories.txt', header=None, names=['id', 'category']) 
df.head()



,id,category
0,1,preaching
1,1,sermon
2,2,wicca
3,2,witchcraft
4,8,general


In [13]:
# lemmatize category as well
df['category'] = df['category'].apply(lambda x: wordnet_lemmatizer.lemmatize(x))

# check if previous step worked
df.head()

In [ ]:
#save model as a file
model.save('text8.model')


In [ ]:
#load model from file
model = gensim.models.Word2Vec.load('text8.model')

In [47]:
# collect all categories for the same id (book)
# lemmatize each category and 
# use word2vec to predict the most similar category (word) to the bag of words
# save result into a file

with open('amazon_new_categories.txt', 'w', encoding="utf8") as file:
    bow = []
    for i in df.id.value_counts().index:
    #for i in df.head(100).id.value_counts().index:
        for category_item in list(set(df[df['id']==i]['category'].tolist())):
            bow += category_item.split()
        lem = []
        for word in bow:
            x = wordnet_lemmatizer.lemmatize(word)
            if x in model.wv.vocab:
                #general is a generic term; thus use the other category instead
                if x != 'general':    
                    lem.append(x)
                if len(lem) == 0:
                    predicted = x
                else:
                    predicted = model.most_similar(lem)[0][0]
                    
        strwrite = str(i) + ',' + str(predicted)
        file.write(strwrite + '\n')
        bow = []
file.close


<function TextIOWrapper.close>

In [41]:
#how the categories are put into a bag of words (bow)
#use sample id: 195777
bow=[]
for word in list(set(df[df['id']==195777]['category'].tolist())):
    bow += word.split()
print(bow)

['cather', 'contemporary', 'austen', 'wharton', 'literary', 'classics', 'classic', 'woolf', 'author', 'general']


In [11]:
#print sample result
print(model.most_similar(['wicca', 'witchcraft'], topn=1))

[('mysticism', 0.8144474029541016)]


In [98]:
#print more sample result
print(model.most_similar(['general', 'logic', 'brain', 'teaser', 'puzzles'], topn=1))

[('randomness', 0.6920949220657349)]


<b> Summary of steps performed on finding similar or duplicate categories: </b> <br>
1) Many categories have the same levels/hierarchy except the last section.  Collect all into a dataframe called df_dup (or dataframe of potential similar or duplicate categories )<br>
2) Clean the end section of these categories by removing non chars like numbers and special symbols <br>
3) Drop rows with NaN values since it has no use. This means the original category value is numeric of symbols <br>
4) Save into a file for processing later on <br>
5) Around 12% potential similar or duplicate categories will be processed later on <br>

<b> Create new categories using word2vec (gensim) supplemented by ntlk lemmatization </b> <br>
1) Collect all category words into a bag of words and remove duplicate words <br>
2) Ensure that these bag of words exists in the corpus else prediction will have an error <br>
  2.1) special handling on the category wording of 'general' since it is a generic category <br>
3) Predict the word similar to the bag of words using word2vec <br>
4) Resulting word is the new category <br>

<b> Sample results: </b> <br>
|Books[283155]|Subjects[1000]|Religion & Spiritual|Wicca and <br>
|Books[283155]|Subjects[1000]|Religion & Spiritual|Witchcraft is predicted as <br>
|Books[283155]|Subjects[1000]|Religion & Spiritual|Mysticism <br>

<b> Recommendation for future research:</b> <br>
1) Use bi-gram to handle words like 'operating systems' rather than treating them as two words 'operating', 'systems' <br>
2) Extend search of similar or duplicated categories from the middle heirarchy/level. <br>
3) Improve the corpus by adding new words in the model. For example; some of the result is funny like 'Cooking' is similar to 'Miso' which is a japanese soup.


<h3> end of solution to category bloat section

<h3> PART B</h3>
B. (Suggested duration: 30 mins)
Give the number crunching a rest! Just think about these problems.
1. Algorithm thinking<br>
How would build the product categorization from scratch, using similar/co-purchased information?<br>
<b> I would collect all words in the product categorization and including the categorization of the similar products to the given product. Then using Naive Bayes, or Support Vector Machine (SVM) or NLTK, I would classify the products based on similar categorization among the given product and it's similar products.
For example: productA is categorized as cat1, cat2, cat3 while similar product is ProductA1 and categorized as cat1, cat2.1, cat5. Then based on similarity coefficient, ProductA is now classified as (0.6cat1, 0.15cat2, 0.15cat.1, .05cat3, .05cat5). 

<br>

</b>
2. Product thinking<br>
Now, put on your 'product thinking' hat.<br>
a. Is it a good idea to show users the categorization hierarchy for items?<br>
<b> Yes, showing users the categorization hierarchy for each items will help them to search for more or other items which are on the same category with what he/she is looking at. This also gives more user engagement on the website and thus increase the probability that this user will purchase an item.
</b>
b. Is it a good idea to show users similar/co-purchased items?<br>
<b> Yes, it is a good idea to show users similar/co-purchased items since it is a quick way to showcase some of the products to the user. It practically increases sales since it will create a good feeling on the customer that it is being personalized for their own needs and increases shopping behavior to check it out or buy more.
</b>
c. Is it a good idea to show users reviews and ratings for items?<br>
<b> Yes, it is a good idea to show users reviews and ratings for items because for some customers that has limited information on the item, they will based their decisions on other customers who bought and used the product. It is also important to display the average rating AND HOW MANY REVIEWS.  A five-star rating with 3 reviewers may not be good compared to 4.5 rating with 100 reviewers.
</b>
d. For each of the above, why? How will you establish the same?<br>
<b>  1. how to show categorization hierarchy for items: Put all the categories for this item and then when user clicks on any category levels, it will display all items related to this item under that category
2. How to show users similar/co-purchased items:  Display the image of the item, price and ratings of all items similar to the item being viewed. Also users can also compared each items side by side according to product description, ratings and price.  
3. How to show users reviews and ratings for items: Similar to what Amazon is currently doing, show users reviews sorted by review date, highest rating, most votes and most helpful.  Also show the reviews with lowest rating so that the user can get the reviews for both good and bad ratings. 
</b>


#End of Report